In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, when, count, lit

In [3]:
ruta = 'C:/Users/omarc/OneDrive/Escritorio/Proyectos VSC/OmarCano/Track Analítico UNI/05_Datos/02_Datos_Procesados'
ruta2 = 'C:/Users/omarc/OneDrive/Escritorio/Proyectos VSC/OmarCano/Track Analítico UNI/05_Datos/01_Datos_Crudos'
rutafin = 'C:/Users/omarc/OneDrive/Escritorio/Proyectos VSC/OmarCano/Track Analítico UNI/05_Datos/03_Datos_Finales'

desertados_df = pd.read_csv(f'{ruta}/um_desercion.csv')
cepre_df = pd.read_csv(f'{ruta}/hm_cepre_uni_limpio.csv')
admisiones_df = pd.read_csv(f'{ruta}/hm_admisiones_uni_limpio.csv')
matriculados_df = pd.read_csv(f'{ruta}/hm_matriculados_uni_limpio.csv')
personas_df = pd.read_csv(f'{ruta2}/hm_personas_demo.csv')
colegios_df = pd.read_csv(f'{ruta2}/hm_colegios.csv')

In [12]:
personas_df
personas_df.to_csv(f'{rutafin}/hm_personas_demo_final.csv', index=False)

In [10]:
admisiones_df['CALIF_FINAL'] = pd.to_numeric(admisiones_df['CALIF_FINAL'], errors='coerce')

admisiones_agrup_df = admisiones_df.groupby('IDHASH').agg(
    MIN_CALIF_ADMIS=('CALIF_FINAL', 'min'),
    MAX_CALIF_ADMIS=('CALIF_FINAL', 'max'),
    PROM_CALIF_ADMIS=('CALIF_FINAL', 'mean'),
    CANT_ADMIS=('IDHASH', 'count')
).reset_index()

admisiones_agrup_df.to_csv(f'{rutafin}/hm_mtz_admisiones_final.csv', index=False)

In [11]:
cepre_agrup_df = cepre_df.groupby('IDHASH').agg(
    MIN_CALIF_CEPRE=('CALIF_FINAL', 'min'),
    MAX_CALIF_CEPRE=('CALIF_FINAL', 'max'),
    PROM_CALIF_CEPRE=('CALIF_FINAL', 'mean'),
    CANT_CEPRE=('IDHASH', 'count')
).reset_index()

cepre_agrup_df.to_csv(f'{rutafin}/hm_mtz_cepre_final.csv', index=False)

In [13]:
desertados_df.columns = [col.upper() for col in desertados_df.columns]
desertados_df.to_csv(f'{rutafin}/hm_desercion_final.csv', index=False)

In [16]:

def categorizar_modalidad(modalidad):
    if 'Terrorismo' in modalidad:
        return 'Victimas del Terrorismo'
    elif 'Escolar' in modalidad:
        return 'Concurso Nacional Escolar'
    elif 'Ingreso Directo' in modalidad:
        return 'Ingreso Directo Cepre-Uni'
    elif 'Titulados' in modalidad:
        return 'Titulados o Graduados'
    elif 'Deportistas Calificados' in modalidad:
        return 'Deportistas Calificados'
    elif 'Convenio' in modalidad:
        return 'Convenios Especiales'
    elif 'Diplomado' in modalidad:
        return 'Diplomados con Bachillerato'
    else:
        return modalidad

matriculados_df['MODALIDAD_AGRUP'] = matriculados_df['MODALIDAD'].apply(categorizar_modalidad)

In [18]:
matriculados_df.columns = [col.upper() for col in matriculados_df.columns]
matriculados_df.to_csv(f'{rutafin}/hm_matriculados_final.csv', index=False)

In [19]:
matriculados_df = matriculados_df[['IDHASH','ANIO','PERIODO', 'TIPO_MATRICULA', 
                                   'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',  
                                   'MODALIDAD_AGRUP', 'METODOLOGIA', 'FACULTAD', 'ESPECIALIDAD',
                                   'CICLO_RELATIVO', 'CODYEAR', 'EDAD', 'FLG_CAMBIO_CARRERA']]

tablon_final_df = matriculados_df.merge(personas_df, on='IDHASH', how='left') \
                           .merge(desertados_df, on='IDHASH', how='left') \
                           .merge(admisiones_agrup_df, on='IDHASH', how='left') \
                           .merge(cepre_agrup_df, on='IDHASH', how='left')

In [20]:
#tablon_final_df['IDHASH'].nunique()
tablon_final_df[tablon_final_df['FLG_ESTUDIANTE'] == 1].groupby('CODYEAR').agg(
    IDHASH_UNICOS=('IDHASH', 'nunique'),  
    DESERTADOS=('DESERCIÓN', lambda x: (x == 1).sum())
)

,IDHASH_UNICOS,DESERTADOS
CODYEAR,,
20161,10110,1154
20162,10023,1069
20171,11771,1178
20172,12262,1323
20181,11905,1045
20182,11250,834
20191,11765,1032
20192,12098,1000
20201,12211,972


In [22]:
variables_seleccionadas = [
    'IDHASH', 'ANIO', 'PERIODO', 'TIPO_MATRICULA', 'DOMICILIO_DEPA',
    'DOMICILIO_PROV', 'MODALIDAD_AGRUP', 'FACULTAD', 'CICLO_RELATIVO', 'CODYEAR', 'EDAD',
    'FLG_CAMBIO_CARRERA', 'COLEGIO_ANIO_EGRESO', 'ANIO_NACIMIENTO',
    'SEXO', 'FLG_ESTUDIANTE', 'DESERCIÓN'
]

# Filtrar las columnas seleccionadas en el dataframe
filtered_df = tablon_final_df[variables_seleccionadas]

# Calcular el fill rate para cada variable
fill_rate = filtered_df.notnull().mean() * 100
fill_rate

IDHASH                 100.000000
ANIO                   100.000000
PERIODO                100.000000
TIPO_MATRICULA         100.000000
DOMICILIO_DEPA          99.238522
DOMICILIO_PROV          88.767029
MODALIDAD_AGRUP        100.000000
FACULTAD               100.000000
CICLO_RELATIVO         100.000000
CODYEAR                100.000000
EDAD                   100.000000
FLG_CAMBIO_CARRERA     100.000000
COLEGIO_ANIO_EGRESO    100.000000
ANIO_NACIMIENTO        100.000000
SEXO                   100.000000
FLG_ESTUDIANTE         100.000000
DESERCIÓN              100.000000
dtype: float64

In [23]:
tablon_final_df.to_csv(f'{rutafin}/tablon_final_df.csv', index=False)